<a href="https://colab.research.google.com/github/Kususumu/SVM4Sketch/blob/master/SVM4Sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Init the Google Drive
from google.colab import drive
drive.mount('/content/drive/')
# Use the below code to show the Drive
#!ls "/content/drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Configure the Environment For SVM4Sketch
# install opencv-python == 3.4.2.16
# Command:: pip install opencv-python == 3.4.2.16
# Command:: pip install opencv-contrib-python==3.4.2.16
# Else you can not use sift
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

In [ ]:
import glob
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import os
import random
import copy

#CLASS:SketchData
class SketchData:
  """SketchData Class

    Attributes:
      init: Class Testor
  """
  
  def __init__(self,path):
    self.self_path = path
    self.default_width = 28

  def set_keypoints(self,size,step):
    self.keypoints = self.create_keypoints(size, size, step)

  def get_keypoints(self):
    return self.keypoints

  def load_dataset(self,type,random = False,random_num_train = 0,random_num_test = 0):
    """
    Function to load all dataset
    Args:
      type:  The type of all data in dataset,Image Type can be jpg|png|and so on.
      random: take the while dataset random as train and test
      train_num: if random take the while dataset, then we need
      test_num: if random take the while dataset, then we need
    """
    self.categories = sorted(glob.glob(self.self_path))
    print('SketchData:categories')
    print(self.categories)
    print('SketchData:categories length: '+str(len(self.categories)))
    print('SketchData:Path')
    print(self.self_path)

    # Get the num of label
    num_label = len(self.categories)

    categories_and_images = []
    num_dataset = 0

    # Read all image in each category
    for cat in self.categories:
      category_name = os.path.basename(os.path.normpath(cat))
      images_in_cat = glob.glob(cat + '/*.'+str(type))
      num_dataset += len(images_in_cat)
      categories_and_images.append((category_name, images_in_cat))

    print('SketchData:num_datasets:'+str(num_dataset))

    # Todo
    # If random the dataset    
    if(random):
      #function random the dataset
      train_set,test_set = self.get_random_train_test_data(categories_and_images,
                        num_label, random_num_train, random_num_test)
      num_train_dataset = int(num_dataset * (random_num_train/(random_num_train + random_num_test)))
      num_test_dataset = int(num_dataset * (random_num_test/(random_num_train + random_num_test)))
      return train_set, num_train_dataset, test_set, num_test_dataset
    else:
      return categories_and_images, num_dataset

  def show_dataset(self,cats_and_images,num_total,num_show):
    """
    Function to show partial dataset
    Args:
      cats_and_images:  A list of label and images path of all data
      num_total: The number of images to show per categories
      num_show: The number of showing per catergories
    """
    # Get the Size of the Matrix
    # Row means the catergories of the Dataset
    rows = len(cats_and_images)
    columns = len(cats_and_images[0])
    print("Dataset Rows:"+str(rows),"Dataset Columns:"+str(columns))
    
    # We get the number of data in each catergories
    num_of_catergories = rows
    num_data_per_catergories = num_total/rows

    num_to_show = num_show
    
    # Here we use the Matplotlib to show the Image
    fig = plt.figure()
    
    # Use for to show every image in catergories
    for i in range(num_to_show):
      for j in range(num_of_catergories):
        fig.add_subplot(num_of_catergories,num_to_show,num_to_show*j+i+1)
        image_now = cv.imread(cats_images[j][1][i])
        imgplot = plt.imshow(image_now)

  def get_training_data(self, sift=True):
    """
    Function to get train dataset
    Args:
      sift:  If the data need sift feature engineering
    """
    self.train_categories_and_images, self.num_train_images , self.test_categories_and_images, self.num_test_images = self.load_dataset('jpg', random = True,random_num_train = 30,random_num_test = 10)

    num_train_images = self.num_train_images
    train_categories_and_images = self.train_categories_and_images

    print("SketchData:num_train_images: "+ str(num_train_images))
    print("SketchData:get_training_data: Loaded %d images" % num_train_images)

    if sift:
      deslen = self.descriptor_length
    else:
      deslen = self.default_width * self.default_width

    # create y_train vector containing the labels as integers
    y_train = np.zeros(num_train_images, dtype=int)

    # x_train matrix containing decriptors as vectors
    x_train = np.zeros((num_train_images, deslen))

    index_img = 0
    index_cat = 0

    for (cat, image_filenames) in train_categories_and_images:
      for image in image_filenames:
        image_read = cv.imread(image, 0)

        if sift:
          des = self.create_sift_descriptors_for_image(image_read)
        else:
          des = image_read

        # each descriptor (set of features) need to be flattened in one vector
        x_train[index_img] = des.flatten()
        y_train[index_img] = index_cat
        index_img = index_img + 1
      index_cat = index_cat + 1

    return x_train, y_train
      
  def get_testing_data(self, sift=True):
    """
    Function to get test dataset
    In This stage, we are not going to random the data cause we 
    have random them in the get_training_data stage
    Args:
      sift:  If the data need sift feature engineering
    Returns:
      x_test: The Feature vector of testing image
      y_test: The label of the testing image
    """
    test_categories_and_images = self.test_categories_and_images
    num_test_images = self.num_test_images

    print("SketchData:num_test_images"+ str(num_test_images))
    print("SketchData:get_testing_data:Loaded %d images" % num_test_images)

    if sift:
      deslen = self.descriptor_length
    else:
      deslen = self.default_width * self.default_width

    # create y_test vector containing the labels as integers
    y_test = np.zeros(num_test_images, dtype=int)

    # x_test matrix containing decriptors as vectors
    x_test = np.zeros((num_test_images, deslen))

    index_img = 0
    index_cat = 0

    for (cat, image_filenames) in test_categories_and_images:
      for image in image_filenames:
        image_read = cv.imread(image, 0)

        if sift:
          des = self.create_sift_descriptors_for_image(image_read)
        else:
          des = image_read

        # each descriptor (set of features) need to be flattened in one vector
        x_test[index_img] = des.flatten()
        y_test[index_img] = index_cat
        index_img = index_img + 1
      index_cat = index_cat + 1

    return x_test, y_test

  
  def get_random_train_test_data(self,dataset,num_label, random_num_train, random_num_test):
    """
    Function to random while dataset
    Args:
      dataset:  the original while dataset
      num_label: the number of Predict value Y
      random_num_train: the number of train data we need in every catergories
      random_num_test: the number of test data we need in every catergories
    Returns:Returns:
      train_set: The train image and label
      test_set: The test image and label
    """
    print(dataset)
    print("Sketchdata:Dataset_random")
    for x in range(num_label):
      random.Random().shuffle(dataset[x][1])
    print(dataset)

    # Get train_set
    print("train_set")
    train_set = copy.deepcopy(dataset)
    print(train_set)
    for x in range(num_label):
      del train_set[x][1][random_num_train:]
    print(train_set)

    # Get test_images
    print("test_set")
    test_set = copy.deepcopy(dataset)
    print(test_set)
    for x in range(num_label):
      del test_set[x][1][:random_num_train]
    print(test_set)

    return train_set,test_set

  def create_sift_descriptors_for_image(self, image):
    # compute one descriptor for each image
    keyp, descr = self.sift.compute(image, self.keypoints)
    return descr
  
  def create_keypoints(self, w, h, keypoint_size):
    """
    Creating keypoints on grid for later image segmentation
    Args:
      w: width of grid
      h: height of grid
      keypoint_size: keypoint size
    Return:
      keypoints:array of kreypoints
    """
    keypoints_list = []

    for x in range(0, w, keypoint_size):
      for y in range(0, h, keypoint_size):
        keypoints_list.append(cv.KeyPoint(x, y, keypoint_size))

    return keypoints_list

  def init_descriptor_keypoint(self):
    """
    Creating sift for sketchdata
    """
    self.keypoints = self.get_keypoints()

    ho4keypoint = 4 * 4 * 8

    self.descriptor_length = ho4keypoint * len(self.keypoints)

    # Initiate SIFT detector
    self.sift = cv.xfeatures2d.SIFT_create()

  

In [ ]:
from sklearn import svm as svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#CLASS:SVM4Sketch
class SVM4Sketch:
  """SVM4Sketch Class

    Attributes:
      sketchdata: To attach and process the sketchdata
      descriptor_length: Use to describe the whole num of Descriptor vector
  """
  def __init__(self,data):
    """
    Init
    Args:
        sketchdata: The Obj to contain train test data
    """
    self.sketchdata = data

    self.scaler = StandardScaler()
 
  def train_model(self, c_range, gamma_range, kernel):
    """
    The function to train the classifier
    Args:
      c_range,gamma_range:parametric for search the model parameter
      kernel: The kernel for SVM
    Returns:
      clf: The model be trained
    """
    x_train, y_train = self.sketchdata.get_training_data(sift=True)
    x_train = self.scaler.fit_transform(x_train)

    # parameters = {'C': c_range, "gamma": gamma_range, 'kernel': ['linear', 'rbf']}
    parameters = {'C': c_range, "gamma": gamma_range}

    # Train a SVM classifier, GridSearchCV in this case
    clf = GridSearchCV(svm.SVC(kernel=kernel), parameters)
    clf.fit(x_train, y_train)

    return clf

  def test_model(self):
    """
    Use this function to test the data
    Returns:
      right_rate: the right rate of the test

    """
    # Test Single Data
    x_test, y_test = self.sketchdata.get_testing_data(sift=True)
    print("SVM4Sketch:x_test images: ")
    print(x_test)
    print("SVM4Sketch:y_test images: ")
    print(y_test)

    i = 0
    num_right = 0
    for image in x_test:
      #print(i)
      # You need 2D array if you want to test you data
      # We use reshape(1,-1) here to get the 2D array
      #print(image)
      image =  np.array(image).reshape(1,-1)
      
      test_descriptor = self.scaler.transform(image)

      test_label = model.best_estimator_.predict(test_descriptor)
      print("SVM4Sketch:This test label is %s, Original label is %s"% (test_label, y_test[i]))
      if test_label == y_test[i]:
        num_right = num_right + 1
      i = i + 1
    
    right_rate = num_right/i
    print("SVM4Sketch:The Right rate of the test is %s" %(right_rate))
    return right_rate


In [ ]:
# The Training And Testing Process
import time

# Calculate time consume
start_time = time.time()

# Path of Dataset
path = "/content/drive/My Drive/TinyPostureDataset/Images/**/"

#
# Training Stage
#
print("Whole Process:Training Stage")

# Import the Data
data = SketchData(path) 
data.set_keypoints(28,28)
data.init_descriptor_keypoint()

# Init a SVM Model
svm_train = SVM4Sketch(data)

# Training Model Paramter4Training
c_range = [10,100,1000]
gamma_range = [0.00001,0.0001,0.001]
model = svm_train.train_model(c_range, gamma_range, kernel="linear")

print("The best parameters are %s with a score of %0.2f" % (model.best_params_, model.best_score_))

# Show Ending Time
print("--- %0.2f minutes ---" % ((time.time() - start_time) / 60))

#
# Testing Stage
#
print("Whole Process:Testing Stage")

# Test the model
# If has model
# model = svm.load_model("")
svm_train.test_model()

print("--- %0.2f minutes ---" % ((time.time() - start_time) / 60))

Whole Process:Training Stage
SketchData:categories
['/content/drive/My Drive/TinyPostureDataset/Images/Tang/', '/content/drive/My Drive/TinyPostureDataset/Images/Zhan/', '/content/drive/My Drive/TinyPostureDataset/Images/Zuo/']
SketchData:categories length: 3
SketchData:Path
/content/drive/My Drive/TinyPostureDataset/Images/**/
SketchData::num_datasets:120
[('Tang', ['/content/drive/My Drive/TinyPostureDataset/Images/Tang/5.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/7.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/10.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/2.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/1.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/4.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/8.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/9.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Tang/3.jpg', '/content/drive/My Drive/TinyPostureDataset/Images/Ta